# Basics

Let's first take a look at what's inside the ``ib_insync`` package:

In [1]:
import ib_insync
print(ib_insync.__all__)

['util', 'Object', 'ContractDetails', 'ContractDescription', 'ComboLeg', 'UnderComp', 'OrderComboLeg', 'OrderState', 'ScannerSubscription', 'SoftDollarTier', 'Execution', 'CommissionReport', 'ExecutionFilter', 'BarList', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'TickAttrib', 'NewsProvider', 'DepthMktDataDescription', 'AccountValue', 'RealTimeBar', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'MktDepthData', 'DOMLevel', 'BracketOrder', 'TradeLogEntry', 'ScanData', 'TagValue', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'OrderCondition', 'ExecutionCondition', 'OperatorCondition', 'MarginCondition', 'ContractCondition', 'TimeCondition', 'PriceCondition', 'PercentChangeCondition', 'VolumeCondition', 'Contract', 'Stock', 'Option', 'Future', 'Forex', 'Index

### Importing
The following two lines are used at the top of all notebooks. The first line imports everything and the second
starts an event loop to keep the notebook live updated:

In [2]:
from ib_insync import *
util.startLoop()

### Connecting
The main player of the whole package is the "IB" class. Let's create an IB instance and connect to a running TWS/IBG application (remember it must have an open API port):

In [3]:
ib = IB()
ib.connect('127.0.0.1', 4002, clientId=1)

ERROR:ib_insync.wrapper:Error 321, reqId 1: Error validating request:-'aS' : cause - ALL account is not supported


<IB connected to 127.0.0.1:4002 clientId=1>

If the connection failed, then double-check the hostname and port. For IB Gateway the default port is 4001. Make sure the clientId is not already in use.

If the connection succeeded, then ``ib`` will be synchronized with TWS/IBG. The "current state" is now available via methods such as ib.positions(), ib.trades(), ib.openTrades(), ib.accountValues() or ib.tickers(). For example, to see the actual net liquidation value:

In [4]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DI246990', tag='NetLiquidationByCurrency', value='1401003', currency='BASE', modelCode='')]

In [5]:
ib.accountValues()

[AccountValue(account='DI246990', tag='AccountCode', value='DI246990', currency='', modelCode=''),
 AccountValue(account='DI246990', tag='AccountOrGroup', value='DI246990', currency='BASE', modelCode=''),
 AccountValue(account='DI246990', tag='AccountOrGroup', value='DI246990', currency='EUR', modelCode=''),
 AccountValue(account='DI246990', tag='AccountOrGroup', value='DI246990', currency='JPY', modelCode=''),
 AccountValue(account='DI246990', tag='AccountOrGroup', value='DI246990', currency='USD', modelCode=''),
 AccountValue(account='DI246990', tag='AccountReady', value='true', currency='', modelCode=''),
 AccountValue(account='DI246990', tag='AccountType', value='INDIVIDUAL', currency='', modelCode=''),
 AccountValue(account='DI246990', tag='AccruedCash', value='214', currency='BASE', modelCode=''),
 AccountValue(account='DI246990', tag='AccruedCash', value='0', currency='EUR', modelCode=''),
 AccountValue(account='DI246990', tag='AccruedCash', value='-22878', currency='JPY', model

This state will automatically be kept in sync with TWS/IBG. So an order fill will be added as soon as it is reported, or account values will be updated as soon as they change in TWS.

### Contracts

Contracts can be specified in different ways:
* The ibapi way, by creating an empty Contract object and setting it's attributes one by one;
* By using Contract and giving the attributes as keyword argument;
* By using the specialized Stock, Option, Future, Forex, Index, CFD, Commodity,
  Bond, FuturesOption, MutualFund or Warrant contracts.

Some examples:

In [6]:
Contract(conId=270639)
Stock('AMD', 'SMART', 'USD')
Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
Forex('EURUSD')
CFD('IBUS30')
Future('ES', '20180921', 'GLOBEX')
Option('SPY', '20170721', 240, 'C', 'SMART')
Bond(secIdType='ISIN', secId='US03076KAA60');

### Sending a request

The IB class has nearly all request methods that the IB API offers. The methods that return a result will block until finished and then return the result. Take for example reqContractDetails:

In [7]:
%%time

contract = Stock('TSLA', 'SMART', 'USD')
cds = ib.reqContractDetails(contract)

CPU times: user 1.93 ms, sys: 2.25 ms, total: 4.18 ms
Wall time: 347 ms


Requesting contract details generally takes about 100 - 300 ms. Let's have a look at the actual
contract details that were sent:

In [8]:
cds

[ContractDetails(summary=Contract(secType='STK', conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS'), marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,POSTONLY,PREOPGRTH,REL,RPI,RTH,SCALE,SCALEODD,SCALERST,SNAPMID,SNAPMKT,SNAPREL,SOLICITED,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,ISE,CHX,ARCA,ISLAND,DRCTEDGE,BEX,BATS,EDGEA,CSFBALGO,JEFFALGO,BYX,IEX,TPLUS1,NYSENAT,PSX', priceMagnifier=1, longName='TESLA INC', industry='Consumer, Cyclical', category='Auto Manufacturers', subcategory='Auto-Cars/Light Trucks', timeZoneId='EST', tradingHours='20180622:0400-2000;20180625:0400-2000', liquidHours='20180622:0930-1600;20180625:0930-1600',

### Logging

The following will put log messages of INFO and higher level under the current active cell:

In [9]:
util.logToConsole()

To see all debug messages (including network traffic):

In [10]:
import logging
util.logToConsole(logging.DEBUG)

DEBUG:ib_insync.client:<<< 61,3,DU774242,10098,BAC,STK,,0.0,,,NYSE,USD,BAC,BAC,-500,23.84453
2018-06-22 13:08:10,542 ib_insync.client DEBUG <<< 61,3,DU774242,10098,BAC,STK,,0.0,,,NYSE,USD,BAC,BAC,-500,23.84453
2018-06-22 13:08:10,542 ib_insync.client DEBUG <<< 61,3,DU774242,10098,BAC,STK,,0.0,,,NYSE,USD,BAC,BAC,-500,23.84453
DEBUG:root:decode <class 'int'> b'3'
2018-06-22 13:08:10,547 root DEBUG decode <class 'int'> b'3'
2018-06-22 13:08:10,547 root DEBUG decode <class 'int'> b'3'
DEBUG:root:decode <class 'str'> b'DU774242'
2018-06-22 13:08:10,557 root DEBUG decode <class 'str'> b'DU774242'
2018-06-22 13:08:10,557 root DEBUG decode <class 'str'> b'DU774242'
DEBUG:root:decode <class 'int'> b'10098'
2018-06-22 13:08:10,562 root DEBUG decode <class 'int'> b'10098'
2018-06-22 13:08:10,562 root DEBUG decode <class 'int'> b'10098'
DEBUG:root:decode <class 'str'> b'BAC'
2018-06-22 13:08:10,569 root DEBUG decode <class 'str'> b'BAC'
2018-06-22 13:08:10,569 root DEBUG decode <class 'str'> b'BAC

2018-06-22 13:08:10,678 root DEBUG decode <class 'str'> b'CASH'
DEBUG:root:decode <class 'str'> b''
2018-06-22 13:08:10,685 root DEBUG decode <class 'str'> b''
2018-06-22 13:08:10,685 root DEBUG decode <class 'str'> b''
DEBUG:root:decode <class 'float'> b'0.0'
2018-06-22 13:08:10,688 root DEBUG decode <class 'float'> b'0.0'
2018-06-22 13:08:10,688 root DEBUG decode <class 'float'> b'0.0'
DEBUG:root:decode <class 'str'> b''
2018-06-22 13:08:10,694 root DEBUG decode <class 'str'> b''
2018-06-22 13:08:10,694 root DEBUG decode <class 'str'> b''
DEBUG:root:decode <class 'str'> b''
2018-06-22 13:08:10,700 root DEBUG decode <class 'str'> b''
2018-06-22 13:08:10,700 root DEBUG decode <class 'str'> b''
DEBUG:root:decode <class 'str'> b''
2018-06-22 13:08:10,704 root DEBUG decode <class 'str'> b''
2018-06-22 13:08:10,704 root DEBUG decode <class 'str'> b''
DEBUG:root:decode <class 'str'> b'USD'
2018-06-22 13:08:10,707 root DEBUG decode <class 'str'> b'USD'
2018-06-22 13:08:10,707 root DEBUG deco

DEBUG:root:decode <class 'str'> b'AUD.USD'
2018-06-22 13:08:10,843 root DEBUG decode <class 'str'> b'AUD.USD'
2018-06-22 13:08:10,843 root DEBUG decode <class 'str'> b'AUD.USD'
DEBUG:root:decode <class 'str'> b'AUD.USD'
2018-06-22 13:08:10,846 root DEBUG decode <class 'str'> b'AUD.USD'
2018-06-22 13:08:10,846 root DEBUG decode <class 'str'> b'AUD.USD'
DEBUG:root:decode <class 'float'> b'150000'
2018-06-22 13:08:10,849 root DEBUG decode <class 'float'> b'150000'
2018-06-22 13:08:10,849 root DEBUG decode <class 'float'> b'150000'
DEBUG:root:decode <class 'float'> b'0.7881105'
2018-06-22 13:08:10,858 root DEBUG decode <class 'float'> b'0.7881105'
2018-06-22 13:08:10,858 root DEBUG decode <class 'float'> b'0.7881105'
INFO:ib_insync.wrapper:position: Position(account='DU774248', contract=Contract(secType='CASH', conId=14433401, symbol='AUD', currency='USD', localSymbol='AUD.USD', tradingClass='AUD.USD'), position=150000.0, avgCost=0.7881105)
2018-06-22 13:08:10,868 ib_insync.wrapper INFO po

2018-06-22 13:08:10,994 ib_insync.wrapper INFO position: Position(account='DU274370', contract=Contract(secType='CASH', conId=14433401, symbol='AUD', currency='USD', localSymbol='AUD.USD', tradingClass='AUD.USD'), position=3000000.0, avgCost=0.7084579)
DEBUG:ib_insync.client:<<< 61,3,DU774244,14433401,AUD,CASH,,0.0,,,,USD,AUD.USD,AUD.USD,-5000000,0.7902382
2018-06-22 13:08:10,997 ib_insync.client DEBUG <<< 61,3,DU774244,14433401,AUD,CASH,,0.0,,,,USD,AUD.USD,AUD.USD,-5000000,0.7902382
2018-06-22 13:08:10,997 ib_insync.client DEBUG <<< 61,3,DU774244,14433401,AUD,CASH,,0.0,,,,USD,AUD.USD,AUD.USD,-5000000,0.7902382
DEBUG:root:decode <class 'int'> b'3'
2018-06-22 13:08:11,001 root DEBUG decode <class 'int'> b'3'
2018-06-22 13:08:11,001 root DEBUG decode <class 'int'> b'3'
DEBUG:root:decode <class 'str'> b'DU774244'
2018-06-22 13:08:11,006 root DEBUG decode <class 'str'> b'DU774244'
2018-06-22 13:08:11,006 root DEBUG decode <class 'str'> b'DU774244'
DEBUG:root:decode <class 'int'> b'14433401

DEBUG:root:decode <class 'int'> b'4725951'
2018-06-22 13:08:11,338 root DEBUG decode <class 'int'> b'4725951'
2018-06-22 13:08:11,338 root DEBUG decode <class 'int'> b'4725951'
DEBUG:root:decode <class 'str'> b'AEO'
2018-06-22 13:08:11,341 root DEBUG decode <class 'str'> b'AEO'
2018-06-22 13:08:11,341 root DEBUG decode <class 'str'> b'AEO'
DEBUG:root:decode <class 'str'> b'STK'
2018-06-22 13:08:11,344 root DEBUG decode <class 'str'> b'STK'
2018-06-22 13:08:11,344 root DEBUG decode <class 'str'> b'STK'
DEBUG:root:decode <class 'str'> b''
2018-06-22 13:08:11,346 root DEBUG decode <class 'str'> b''
2018-06-22 13:08:11,346 root DEBUG decode <class 'str'> b''
DEBUG:root:decode <class 'float'> b'0.0'
2018-06-22 13:08:11,348 root DEBUG decode <class 'float'> b'0.0'
2018-06-22 13:08:11,348 root DEBUG decode <class 'float'> b'0.0'
DEBUG:root:decode <class 'str'> b''
2018-06-22 13:08:11,352 root DEBUG decode <class 'str'> b''
2018-06-22 13:08:11,352 root DEBUG decode <class 'str'> b''
DEBUG:root

...

2018-06-22 13:41:38,512 root DEBUG decode <class 'str'> b'USD'
DEBUG:root:decode <class 'str'> b'AAPL'
2018-06-22 13:41:38,514 root DEBUG decode <class 'str'> b'AAPL'
2018-06-22 13:41:38,514 root DEBUG decode <class 'str'> b'AAPL'
DEBUG:root:decode <class 'str'> b'NMS'
2018-06-22 13:41:38,516 root DEBUG decode <class 'str'> b'NMS'
2018-06-22 13:41:38,516 root DEBUG decode <class 'str'> b'NMS'
DEBUG:root:decode <class 'float'> b'200'
2018-06-22 13:41:38,518 root DEBUG decode <class 'float'> b'200'
2018-06-22 13:41:38,518 root DEBUG decode <class 'float'> b'200'
DEBUG:root:decode <class 'float'> b'185.2200012'
2018-06-22 13:41:38,520 root DEBUG decode <class 'float'> b'185.2200012'
2018-06-22 13:41:38,520 root DEBUG decode <class 'float'> b'185.2200012'
DEBUG:root:decode <class 'float'> b'37044.0'
2018-06-22 13:41:38,525 root DEBUG decode <class 'float'> b'37044.0'
2018-06-22 13:41:38,525 root DEBUG decode <class 'float'> b'37044.0'
DEBUG:root:decode <class 'float'> b'174.95'
2018-06-22 

INFO:root:ANSWER updateAccountTime {'timeStamp': '13:41'}
2018-06-22 13:41:38,832 root INFO ANSWER updateAccountTime {'timeStamp': '13:41'}
2018-06-22 13:41:38,832 root INFO ANSWER updateAccountTime {'timeStamp': '13:41'}
DEBUG:ib_insync.client:<<< 7,8,15016059,USD,CASH,,0,0,,IDEALPRO,JPY,USD.JPY,USD.JPY,400000,109.9465027,4.397860107E7,107.25325,1077301.07,0.0,DI246990
2018-06-22 13:41:38,835 ib_insync.client DEBUG <<< 7,8,15016059,USD,CASH,,0,0,,IDEALPRO,JPY,USD.JPY,USD.JPY,400000,109.9465027,4.397860107E7,107.25325,1077301.07,0.0,DI246990
2018-06-22 13:41:38,835 ib_insync.client DEBUG <<< 7,8,15016059,USD,CASH,,0,0,,IDEALPRO,JPY,USD.JPY,USD.JPY,400000,109.9465027,4.397860107E7,107.25325,1077301.07,0.0,DI246990
DEBUG:root:decode <class 'int'> b'8'
2018-06-22 13:41:38,838 root DEBUG decode <class 'int'> b'8'
2018-06-22 13:41:38,838 root DEBUG decode <class 'int'> b'8'
DEBUG:root:decode <class 'int'> b'15016059'
2018-06-22 13:41:38,840 root DEBUG decode <class 'int'> b'15016059'
2018-0

2018-06-22 13:41:38,937 root DEBUG decode <class 'float'> b'66.435997'
2018-06-22 13:41:38,937 root DEBUG decode <class 'float'> b'66.435997'
DEBUG:root:decode <class 'float'> b'465.05'
2018-06-22 13:41:38,939 root DEBUG decode <class 'float'> b'465.05'
2018-06-22 13:41:38,939 root DEBUG decode <class 'float'> b'465.05'
DEBUG:root:decode <class 'float'> b'38.8378'
2018-06-22 13:41:38,941 root DEBUG decode <class 'float'> b'38.8378'
2018-06-22 13:41:38,941 root DEBUG decode <class 'float'> b'38.8378'
DEBUG:root:decode <class 'float'> b'193.19'
2018-06-22 13:41:38,945 root DEBUG decode <class 'float'> b'193.19'
2018-06-22 13:41:38,945 root DEBUG decode <class 'float'> b'193.19'
DEBUG:root:decode <class 'float'> b'0.0'
2018-06-22 13:41:38,946 root DEBUG decode <class 'float'> b'0.0'
2018-06-22 13:41:38,946 root DEBUG decode <class 'float'> b'0.0'
DEBUG:root:decode <class 'str'> b'DI246990'
2018-06-22 13:41:38,948 root DEBUG decode <class 'str'> b'DI246990'
2018-06-22 13:41:38,948 root DEB

DEBUG:root:decode <class 'int'> b'8'
2018-06-22 13:41:39,219 root DEBUG decode <class 'int'> b'8'
2018-06-22 13:41:39,219 root DEBUG decode <class 'int'> b'8'
DEBUG:root:decode <class 'int'> b'279396694'
2018-06-22 13:41:39,221 root DEBUG decode <class 'int'> b'279396694'
2018-06-22 13:41:39,221 root DEBUG decode <class 'int'> b'279396694'
DEBUG:root:decode <class 'str'> b'ES'
2018-06-22 13:41:39,222 root DEBUG decode <class 'str'> b'ES'
2018-06-22 13:41:39,222 root DEBUG decode <class 'str'> b'ES'
DEBUG:root:decode <class 'str'> b'FUT'
2018-06-22 13:41:39,223 root DEBUG decode <class 'str'> b'FUT'
2018-06-22 13:41:39,223 root DEBUG decode <class 'str'> b'FUT'
DEBUG:root:decode <class 'str'> b'20180921'
2018-06-22 13:41:39,225 root DEBUG decode <class 'str'> b'20180921'
2018-06-22 13:41:39,225 root DEBUG decode <class 'str'> b'20180921'
DEBUG:root:decode <class 'float'> b'0'
2018-06-22 13:41:39,228 root DEBUG decode <class 'float'> b'0'
2018-06-22 13:41:39,228 root DEBUG decode <class 

2018-06-22 13:41:39,284 ib_insync.client DEBUG <<< 6,2,CashBalance,378745,BASE,DI246990
DEBUG:ib_insync.client:<<< 6,2,TotalCashBalance,378745,BASE,DI246990
2018-06-22 13:41:39,286 ib_insync.client DEBUG <<< 6,2,TotalCashBalance,378745,BASE,DI246990
2018-06-22 13:41:39,286 ib_insync.client DEBUG <<< 6,2,TotalCashBalance,378745,BASE,DI246990
DEBUG:ib_insync.client:<<< 6,2,AccruedCash,213,BASE,DI246990
2018-06-22 13:41:39,288 ib_insync.client DEBUG <<< 6,2,AccruedCash,213,BASE,DI246990
2018-06-22 13:41:39,288 ib_insync.client DEBUG <<< 6,2,AccruedCash,213,BASE,DI246990
DEBUG:ib_insync.client:<<< 6,2,FuturesPNL,6050,BASE,DI246990
2018-06-22 13:41:39,292 ib_insync.client DEBUG <<< 6,2,FuturesPNL,6050,BASE,DI246990
2018-06-22 13:41:39,292 ib_insync.client DEBUG <<< 6,2,FuturesPNL,6050,BASE,DI246990
DEBUG:ib_insync.client:<<< 6,2,NetLiquidationByCurrency,1401694,BASE,DI246990
2018-06-22 13:41:39,294 ib_insync.client DEBUG <<< 6,2,NetLiquidationByCurrency,1401694,BASE,DI246990
2018-06-22 13:4

### Disconnecting

The following will disconnect ``ib`` and clear all its state:

In [ ]:
ib.disconnect()

***